In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

# **Load and Prepare the dataset**

In [5]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]


README.md:   0%|          | 0.00/371 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3 [00:00<?, ? examples/s]

# **Create a Retriever Tool**

In [6]:
!pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.1 MB/s eta 0:00:00


In [7]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [8]:
!pip install rank_bm25

In [9]:
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

class GuestInfoRetrieverTool(Tool):
    name = "guest_info_retriever"
    description = "Retrieves detailed information about gala guests based on their name or relation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The name or relation of the guest you want information about."
        }
    }
    output_type = "string"

    def __init__(self, docs):
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)

    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in results[:3]])
        else:
            return "No matching guest information found."

# Initialize the tool
guest_info_tool = GuestInfoRetrieverTool(docs)

# **Integrate the tool with Alfred**

In [10]:
from smolagents import CodeAgent, HfApiModel

# Initialize the Hugging Face model
model = HfApiModel()

# Create Alfred, our gala agent, with the guest info tool
alfred = CodeAgent(tools=[guest_info_tool], model=model)

# Example query Alfred might receive during the gala
response = alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about our guest named 'Lady Ada Lovelace'.                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  lady_ada_info = guest_info_retriever(query="Lady Ada Lovelace")                                                  
  print(lady_ada_info)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

<ipython-input-9-9a4372193305>:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = self.retriever.get_relevant_documents(query)


Execution logs:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(lady_ada_info)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

[Step 2: Duration 2.91 seconds| Input tokens: 4,442 | Output tokens: 122]

🎩 Alfred's Response:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com


# **Provide access to the web**

In [11]:
from smolagents import DuckDuckGoSearchTool

# Initialize the DuckDuckGo search tool
search_tool = DuckDuckGoSearchTool()

# Example usage
results = search_tool("Who's the current President of France?")
print(results)

## Search Results

[President of France - Wikipedia](https://en.wikipedia.org/wiki/President_of_France)
The president of France, officially the president of the French Republic (French: Président de la République française), is the executive head of state of France, and the commander-in-chief of the French Armed Forces.As the presidency is the supreme magistracy of the country, the position is the highest office in France. The powers, functions and duties of prior presidential offices, in ...

[Emmanuel Macron - Wikipedia](https://en.wikipedia.org/wiki/Emmanuel_Macron)
Emmanuel Jean-Michel Frédéric Macron ([emanɥɛl makʁɔ̃] ⓘ; born 21 December 1977) is a French politician who has served as President of France and Co-Prince of Andorra since 2017. He was Minister of Economics, Industry and Digital Affairs under President François Hollande between 2014-16. He has been a member of Renaissance since he founded it in 2016.

[Presidency of Emmanuel Macron - Wikipedia](https://en.wikipedia.org/

# **A custom tool for weather information to schedule the fireworks**

In [12]:
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather information for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        # Dummy weather data
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        # Randomly select a weather condition
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = WeatherInfoTool()

# **Creating a Hub Stats Tool for Influential AI Builders**

In [13]:
from smolagents import Tool
from huggingface_hub import list_models

class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Fetches the most downloaded model from a specific author on the Hugging Face Hub."
    inputs = {
        "author": {
            "type": "string",
            "description": "The username of the model author/organization to find models from."
        }
    }
    output_type = "string"

    def forward(self, author: str):
        try:
            # List models from the specified author, sorted by downloads
            models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

            if models:
                model = models[0]
                return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
            else:
                return f"No models found for author {author}."
        except Exception as e:
            return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = HubStatsTool()

# Example usage
print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/esmfold_v1 with 24,680,160 downloads.


# **Integrating Tools with Alfred**

In [14]:
from smolagents import CodeAgent, HfApiModel

# Initialize the Hugging Face model
model = HfApiModel()

# Create Alfred with all the tools
alfred = CodeAgent(
    tools=[search_tool, weather_info_tool, hub_stats_tool],
    model=model
)

# Example query Alfred might receive during the gala
response = alfred.run("What is Facebook and what's their most popular model?")

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Facebook and what's their most popular model?                                                           │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  facebook_info = web_search(query="What is Facebook?")                                                            
  print("Facebook Info:", facebook_info)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Facebook Info: ## Search Results

[Facebook Explained: Origins, Why People Like It, and Key 
Features](https://www.lifewire.com/what-is-facebook-3486391)
Facebook allows you to maintain a friends list and choose privacy settings to tailor who can see content on your 
profile.; Facebook allows you to upload photos and maintain photo albums that can be shared with your friends.; 
Facebook supports interactive online chat and the ability to comment on your friend's profile pages to keep in 
touch, share information or to say "hi."

[Facebook - Wikipedia](https://en.wikipedia.org/wiki/Facebook)
Facebook said its investigation found a Pakistani military link, along with a mix of real accounts of ISPR 
employees, and a network of fake accounts created by them that have been operating military fan pages, general 
interest pages but were posting content about Indian politics while trying to conceal their identity. [480]

[Facebook | Overview, History, Controversies, & Facts | Britannica 
Money](https://www.britannica.com/money/Facebook)
Facebook is an online platform that connects people through profiles, groups, messages, and ads. Learn about its 
origins, features, controversies, and impact on society and business.

[What is Facebook? - TechTarget](https://www.techtarget.com/whatis/definition/Facebook)
Facebook is a popular free social networking website that enables users to share information and photos and keep in
touch with family and friends.

[Facebook: What is Facebook? - GCFGlobal.org](https://edu.gcfglobal.org/en/facebook101/what-is-facebook/1/)
Learn what Facebook is, how it works, and why people use it. Find out how to connect and share with family and 
friends online, and how to adjust your privacy settings.

[How Facebook Works - 
HowStuffWorks](https://computer.howstuffworks.com/internet/social-networking/networks/facebook.htm)
Facebook is a Web site that lets you connect with friends, share information and join networks. Learn how to create
a profile, find friends, use applications and access Facebook on mobile devices.

[Explained: What is Facebook? - Webwise.ie](https://www.webwise.ie/parents/explained-what-is-facebook-2/)
Facebook is a social networking website that allows users to connect, share and communicate online. Learn about its
history, popularity, features, risks and how to stay safe on it.

[What is Facebook? - Digital 
Unite](https://www.digitalunite.com/technology-guides/social-networking-blogs/facebook-and-instagram/what-facebook)
• Facebook is free to join and free to use. • It can be used by anyone wherever they are in the world as long as 
they have internet access. • Facebook users can share messages, videos links, photographs and websites. • Facebook 
users can join groups or follow people/hobbies that interest them • You can search by name or email to find ...

[What is Facebook? An Explainer Guide for Internet Newbies - 
groovyPost](https://www.groovypost.com/howto/what-is-facebook-an-explainer-guide/)
What is Facebook? Facebook is a social networking website where like-minded people can get together to discuss 
their mutual interests. People can also "follow" friends and family for their ...

[What is Facebook? - Pocket Tactics](https://www.pockettactics.com/what-is-facebook)
Facebook professional mode is for creators and businesses to 'reach their full potential on Facebook', allowing you
to earn money and view insights so you can better understand what types of posts resonate well with your community.
In professional mode, you can unlock revenue opportunities for your business' page, access useful tools to ...

Out: None

[Step 1: Duration 6.17 seconds| Input tokens: 2,136 | Output tokens: 78]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Facebook is a social networking platform, and their most popular model on the Hugging Face Hub is  
  facebook/esmfold_v1.")                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Facebook is a social networking platform, and their most popular model on the Hugging Face Hub 
is facebook/esmfold_v1.

[Step 3: Duration 4.51 seconds| Input tokens: 8,437 | Output tokens: 260]

🎩 Alfred's Response:
Facebook is a social networking platform, and their most popular model on the Hugging Face Hub is facebook/esmfold_v1.


# **Assembling Alfred: The Complete Agent**

In [16]:
!pip install tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tools: filename=tools-0.1.9-py3-none-any.whl size=46730 sha256=7adb15508d7532e005c93e2a73e83b92c6e63a518236cf0c83eb904c29e23eac
  Stored in directory: /root/.cache/pip/wheels/bc/d8/9d/52ad6058db295741fe0b776c0fcfdb6670036acab59ce4ccfd
  Created wheel for pytils: filename=pytils-0.4.3-py3-none-any.whl size=32806 sha256=c7058095fc63065970b614b7ff80401d1d49588d5d82b579a6d9403c19340ab4
  Stored in directory: /root/.cache/pip/wheels/3e/a7/be/135c0d4eaa74b54f43b5b0e0b30284b1c2081fe0581424408a
Successfully built tools pytils


In [21]:
!pip install retriever

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00


In [35]:
# Initialize the Hugging Face model
model = HfApiModel()

# Initialize the web search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the weather tool
weather_info_tool = WeatherInfoTool()

# Initialize the Hub stats tool
hub_stats_tool = HubStatsTool()

# Load the guest dataset and initialize the guest info tool
#guest_info_tool = load_guest_dataset()

# Create Alfred with all the tools
alfred = CodeAgent(
    tools=[guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,  # Add any additional base tools
    planning_interval=3   # Enable planning every 3 steps
)

In [36]:
query = "Tell me about 'Lady Ada Lovelace'"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about 'Lady Ada Lovelace'                                                                               │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- We are asked to gather information about 'Lady Ada Lovelace'.

### 1.2. Facts to look up
- Comprehensive biographical details about Lady Ada Lovelace
    - Where to find: Use `web_search("biography of Ada Lovelace")` to get relevant pages.
    - Relevant fields: Birth date, birthplace, death date, family background, education, notable achievements.
- Role and contributions in the field of computing
    - Where to find: Use `web_search("Ada Lovelace contributions to computing")` to find relevant pages.
    - Relevant fields: Specific technologies or concepts she worked on, impact on the field.
- Personal life events
    - Where to find: Use `web_search("Ada Lovelace personal life")` to find relevant pages.
    - Relevant fields: Significant relationships, social circles, personal beliefs.

### 1.3. Facts to derive
- Timeline of key life events including births, marriages, deaths and milestones in her career.
    - Reasoning: Compile derived from dates and events found in her biography.
- Summary of her achievements and their significance.
    - Reasoning: Aggregate information about her contributions to computing, cross-referencing different sources 
for consistency.

## 2. Plan
1. Perform a web search for Lady Ada Lovelace's biography.
2. Perform a web search for Lady Ada Lovelace's contributions to computing.
3. Perform a web search for Lady Ada Lovelace's personal life.
4. Compile a timeline of key life events.
5. Create a summary of her achievements and their significance.
6. Combine everything into a comprehensive overview of Lady Ada Lovelace.
7. Provide the final answer.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Lady Ada Lovelace's biography                                                                       
  bio_info = web_search("biography of Ada Lovelace")                                                               
  print("Biography Info: ", bio_info)                                                                              
                                                                                                                   
  # Search for Lady Ada Lovelace's contributions to computing                                                      
  computing_info = web_search("Ada Lovelace contributions to computing")                                           
  print("Computing Info: ", computing_info)                                                                        
                                                                                                                   
  # Search for Lady Ada Lovelace's personal life                                                                   
  personal_life_info = web_search("Ada Lovelace personal life")                                                    
  print("Personal Life Info: ", personal_life_info)                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Biography Info:  ## Search Results

[Ada Lovelace | Biography, Computer, & Facts | Britannica](https://www.britannica.com/biography/Ada-Lovelace)
Ada Lovelace, English mathematician, an associate of Charles Babbage, for whose digital computer prototype, the 
Analytical Engine, she created a program in 1843. She has been called the first computer programmer. Ada Lovelace 
Day, the second Tuesday in October, honors women's contributions to science and technology.

[Ada Lovelace - Wikipedia](https://en.wikipedia.org/wiki/Ada_Lovelace)
Augusta Ada King, Countess of Lovelace (née Byron; 10 December 1815 - 27 November 1852), also known as Ada 
Lovelace, was an English mathematician and writer chiefly known for her work on Charles Babbage's proposed 
mechanical general-purpose computer, the Analytical Engine.She was the first to recognise that the machine had 
applications beyond pure calculation.

[Ada Lovelace - Quotes, Children & Facts - Biography](https://www.biography.com/scholars-educators/ada-lovelace)
English mathematician Ada Lovelace, the daughter of poet Lord Byron, has been called "the first computer 
programmer" for writing an algorithm for a computing machine in the mid-1800s.

[Biography of Ada Lovelace, First Computer Programmer - 
ThoughtCo](https://www.thoughtco.com/ada-lovelace-biography-5113321)
Ada Lovelace (born Augusta Ada Byron; December 10, 1815- November 27, 1852) was an English mathematician who has 
been called the first computer programmer for writing an algorithm, or a set of operating instructions, for the 
early computing machine built by Charles Babbage in 1821. As the daughter of the famed English Romantic poet Lord 
Byron, her life has been characterized as a constant ...

[Ada Lovelace - Biography, Facts and Pictures - Famous Scientists](https://www.famousscientists.org/ada-lovelace/)
Ada Lovelace was born in London, England, UK on December 10, 1815. She was named Augusta Ada Byron. Her surname 
changed after she married. Her father was the brilliant, yet notorious poet Lord Byron (mad, bad and dangerous to 
know!) and her mother was Anne Isabella Milbanke. Her father was one of the greats of poetry, but his personality 
was ...

[Ada Lovelace - National Geographic Kids](https://kids.nationalgeographic.com/history/article/ada-lovelace)
Ada Lovelace excelled at them—and became what some say is the world's first computer programmer. Born in England on
December 10, 1815, Ada was the daughter of the famous poet Lord George Byron and his wife, Lady Anne Byron. Her 
father left the family just weeks after Ada's birth, but her mother insisted that her daughter have expert ...

[Ada Lovelace (1815 - 1852) - Biography - MacTutor History of 
...](https://mathshistory.st-andrews.ac.uk/Biographies/Lovelace/)
Biography Augusta Ada Byron's father was the famous poet Lord George Gordon Byron and her mother was Anne Isabelle 
Milbanke.Ada's parents married on 2 January 1815 but separated on 16 January 1816, a month after she was born.On 25
April 1816 Lord Byron went abroad and Ada never saw her father again. Lord Byron never returned to England and died
in Greece when Ada was eight years old.

[Ada Lovelace Biography - Childhood, Life Achievements & 
Timeline](https://www.thefamouspeople.com/profiles/ada-lovelace-6234.php)
Ada Lovelace was an English mathematician, known as the world's first computer programmer. This biography provides 
detailed information about her childhood, profile, career and timeline

[Who was Ada? - Ada Lovelace Day](https://findingada.com/about/who-was-ada/)
Who was Ada Lovelace? By Sydney Padua, author of The Thrilling Adventures of Lovelace & Babbage.. Read the longer 
biography of Ada Lovelace by Suw Charman-Anderson, taken from our book, A Passion for Science: Stories of Discovery
and Invention, but for a short overview of her life and achievements, read on!. The woman most often known as 'Ada 
Lovelace' was born Ada Gordon in 1815, sole ...

[Ada Lovelace - Mathematics

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extracting key details from biography information                                                              
  biography_details = {                                                                                            
      "full_name": "Augusta Ada King, Countess of Lovelace",                                                       
      "birth_date": "10 December 1815",                                                                            
      "birth_place": "London, England",                                                                            
      "death_date": "27 November 1852",                                                                            
      "father": "Lord George Gordon Byron, a famous poet",                                                         
      "mother": "Anne Isabella Milbanke",                                                                          
      "marriage": "Married William King in 1835. He became the Earl of Lovelace three years later.",               
      "legacy": "Often referred to as the world's first computer programmer."                                      
  }                                                                                                                
                                                                                                                   
  # Extracting key details from computing contributions information                                                
  computing_contributions = {                                                                                      
      "primary_contribution": "She created a program in 1843 for Charles Babbage's Analytical Engine, which was a  
  prototype for a digital computer.",                                                                              
      "other_contributions": [                                                                                     
          "Translated and annotated a French article by Luigi Federico Menabrea about Babbage's Analytical         
  Engine.",                                                                                                        
          "Appended Notes G to the translation, where she detailed instructions on how the Analytical Engine       
  could be used to calculate Bernoulli numbers, effectively writing the world's first computer algorithm."         
      ],                                                                                                           
      "legacy": "Ada Lovelace Day honors women's contributions to science and technology."                         
  }                                                                                                                
                                                                                                                   
  # Extracting key details from personal life information                                                          
  personal_life_details = {                                                                                        
      "family": "Ada was the only legitimate child of Lord Byron. Her parents separated when she was young.",      
      "early_life": "Her mother prioritized logical subjects like math and science in Ada's education, which were  
  believed to protect her from her father's eccentricities.",                                                      
      "adult_life": "Ada Lovelace made significant contributions to early computing while being a wife and         
  mother."                                                                                                         
  }                                                                                                                
                                                         

Out - Final answer: 
**Ada Lovelace Overview**

**Biography:**
- **Name:** Augusta Ada King, Countess of Lovelace
- **Birth Date:** 10 December 1815
- **Birth Place:** London, England
- **Death Date:** 27 November 1852
- **Parentage:** Daughter of Lord George Gordon Byron, a famous poet and Anne Isabella Milbanke
- **Marriage:** Married William King in 1835. He became the Earl of Lovelace three years later.
- **Legacy:** Often referred to as the world's first computer programmer.

**Computing Contributions:**
- **Primary Contribution:** She created a program in 1843 for Charles Babbage's Analytical Engine, which was a 
prototype for a digital computer.
- **Other Contributions:**
    - Translated and annotated a French article by Luigi Federico Menabrea about Babbage's Analytical Engine.
    - Detailed instructions on how the Analytical Engine could be used to calculate Bernoulli numbers, effectively 
writing the world's first computer algorithm.
- **Legacy:** Ada Lovelace Day honors women's contributions to science and technology.

**Personal Life:**
- **Family:** Ada was the only legitimate child of Lord Byron. Her parents separated when she was young.
- **Early Life:** Her mother prioritized logical subjects like math and science in Ada's education, which were 
believed to protect her from her father's eccentricities.
- **Adult Life:** Ada Lovelace made significant contributions to early computing while being a wife and mother.

[Step 2: Duration 40.29 seconds| Input tokens: 8,559 | Output tokens: 872]

🎩 Alfred's Response:

**Ada Lovelace Overview**

**Biography:**
- **Name:** Augusta Ada King, Countess of Lovelace
- **Birth Date:** 10 December 1815
- **Birth Place:** London, England
- **Death Date:** 27 November 1852
- **Parentage:** Daughter of Lord George Gordon Byron, a famous poet and Anne Isabella Milbanke
- **Marriage:** Married William King in 1835. He became the Earl of Lovelace three years later.
- **Legacy:** Often referred to as the world's first computer programmer.

**Computing Contributions:**
- **Primary Contribution:** She created a program in 1843 for Charles Babbage's Analytical Engine, which was a prototype for a digital computer.
- **Other Contributions:**
    - Translated and annotated a French article by Luigi Federico Menabrea about Babbage's Analytical Engine.
    - Detailed instructions on how the Analytical Engine could be used to calculate Bernoulli numbers, effectively writing the world's first computer algorithm.
- **Legacy:** Ada Lovelace Day honors wom

In [37]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris tonight? Will it be suitable for our fireworks display?                        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- The location of interest is Paris.
- The weather is wanted for tonight.
- The suitability of the weather for a fireworks display is required.

### 1.2. Facts to look up
- The current date and time to determine "tonight."  
  - Source: Any calendar or clock, system time by default.
- Weather forecast for Paris tonight.  
  - Source: A weather API or weather forecasts website (e.g., weather.com or a dedicated API like OpenWeatherMap).
- Typical weather conditions suitable for fireworks displays (clear skies, no heavy rain).  
  - Source: General knowledge on fireworks requirements or guidelines w.r.t. weather.

### 1.3. Facts to derive
- Whether the conditions retrieved from the weather forecast align with the typical conditions suitable for 
fireworks displays.

---

## 2. Plan
1. Obtain the current date and time.
2. Retrieve the weather forecast for Paris for the current evening.
3. Determine the typical weather conditions suitable for fireworks displays.
4. Compare the Paris weather forecast with the typical conditions suitable for fireworks.
5. Provide the verdict on the weather's suitability for tonight's fireworks display.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  weather_info_paris = weather_info(location="Paris")                                                              
  print(weather_info_paris)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Weather in Paris: Rainy, 15°C

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The weather in Paris is rainy, which is not suitable for a fireworks display.")                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The weather in Paris is rainy, which is not suitable for a fireworks display.

[Step 2: Duration 4.34 seconds| Input tokens: 5,248 | Output tokens: 154]

🎩 Alfred's Response:
The weather in Paris is rainy, which is not suitable for a fireworks display.


In [38]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ One of our guests is from Qwen. What can you tell me about their most popular model?                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts Survey

### 1.1. Facts given in the task
- One of our guests is from Qwen.
- We need to learn about their most popular model.

### 1.2. Facts to look up
- Information about the guest from Qwen.
    - **Source:** `guest_info_retriever(query: string)`
- The most downloaded model from the guest (assuming Qwen as the author).
    - **Source:** `hub_stats(author: string)`

### 1.3. Facts to derive
- The full profile or description of the most popular model from the guest.

## 2. Plan
1. Use `guest_info_retriever` to gather information about the guest from Qwen.
2. Identify the guest's username or organization name on the Hugging Face Hub from the gathered information.
3. Use `hub_stats` to find the most downloaded model from the identified guest (author).
4. Retrieve the full profile or detailed information about the most popular model using the Hugging Face Hub stats.
5. Format and provide the final details of the most popular model using `final_answer`.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  guest_info = guest_info_retriever(query="Qwen")                                                                  
  print(guest_info)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_popular_model_info = hub_stats(author="Qwen")                                                               
  print(most_popular_model_info)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The most downloaded model by Qwen is Qwen/Qwen2.5-7B-Instruct with 3,141,716 downloads.

Out: None

[Step 3: Duration 5.88 seconds| Input tokens: 9,508 | Output tokens: 254]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-680c6b23-17668a0c4733e78201ece953;8596fae6-6d0d-4b3c-8f4a-6be782113aa4)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [39]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for │
│ this conversation?                                                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-680c6b32-228f0b487868932e5e6ac5ff;8c397a6a-76a1-4272-ba94-0e85a6894981)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [40]:
# Create Alfred with conversation memory
alfred_with_memory = CodeAgent(
    tools=[guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,
    planning_interval=3
)

# First interaction
response1 = alfred_with_memory.run("Tell me about Lady Ada Lovelace.")
print("🎩 Alfred's First Response:")
print(response1)

# Second interaction (referencing the first)
response2 = alfred_with_memory.run("What projects is she currently working on?", reset=False)
print("🎩 Alfred's Second Response:")
print(response2)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about Lady Ada Lovelace.                                                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-680c6b36-1a319976025720806411c7ca;8adfdb50-6a48-478b-81ab-f3f040cf7e31)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.